# Language Warmup Full Model

## Data Cleaning

### Getting the Cleaning Tools Ready

To clean the dataset, we will be using Pandas, NumPy, RegEx, NLTK and an autocorrect library. There are some custom solutions here that will be explained further via inline comments. Here, we are just importing the libraries and even selecting specific modules from some of them.

In [1]:
import pandas as pd
import numpy as np
import string
import re
import nltk

# 'spell' will be our spell checker and corrector for cleaning purposes
from autocorrect import spell

# A stopword for our purposes is a word that doesn't add a lot of insight to the sentiment of a sentence
nltk.download('stopwords')
nltk.download('wordnet')

# A lemmatizer is a way for us to find the root of a word. 
# Using this, 'grows', 'grew', and 'grown' all evaluate to 'grow'.
lm = nltk.WordNetLemmatizer()

stopword = nltk.corpus.stopwords.words('english')

# We are removing the word 'not' to avoid a situation where for example 'not good' == 'good' 
stopword = [word for word in stopword if word != 'not']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Troy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Troy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Reading in the Data

These two lines of code make it really easy to manipulate the dataset by organizing it into labelled columns and rows, called a dataframe. We can create functions to manipulate specific columns, and for our purposes, we'd like to play around with the reviews only. To do that, first we must read in the data. We can use Pandas' **pd.read_csv** as seen below to read a specific file, **Yelp.txt**. We just have to indicate that each colummn is separated by a tab, or **\t** which we have done. **Header** is set to None because we want to read the file from its first line. Finally, encoding has been set to **Latin1**. This is because of a decoding error that was thrown at us. Encoding in another character set seemed to fix the problem and create no new ones, so we didn't look back. 

Now that the dataset is read in, we can name our columns as they appear from left to right by using the **.columns** method in Pandas.

In [2]:
yelpDataset = pd.read_csv('Yelp.txt', sep='\t', header=None, encoding='latin-1')
yelpDataset.columns = ['review', 'sentiment']

### Creating the Cleaning Functions

With our data set up the way it is, our cleaning is made fairly simple. Each of these functions 'reads' each review and then executes its function on each of them. The purpose of each function will be explained with inline comments.

In [3]:
# This function takes each review and breaks it up into its individual words
def tokenize(text):
    tokens = re.split('\W+', text) # '\W' will split on any non-word characters such as a whitespace, comma or period
    return tokens

# This function removes all characters from 'review' that are NOT alaphabetical (special characters, punctuation etc)
def onlyAlpha(tokenizedList):
    text = [word for word in tokenizedList if word.isalpha()]
    return text

# This function removes any instance of a stopword in each review
def noStop(tokenizedList):
    text = [word for word in tokenizedList if word not in stopword]
    return text

# This function checks for and corrects spelling errors in each word of each review
def spellCheck(tokenizedList):
    text = [spell(word) for word in tokenizedList]
    return text

# This function uses the lemmatizer mentioned earlier on each word of each review
def lemmatize(tokenizedList):
    text = ' '.join([lm.lemmatize(word) for word in tokenizedList]) # .join is used at the end to return the tokenized
    return text                                                     # reviews and returning them to full sentences




### Applying the Cleaning Functions

To apply each function, we decided to use lambda functions. Using lambda functions is beneficial here because we are performing a fairly specific sequence of tasks. On the left of each expression we are naming a new column (usually to reflect a change), and on the right, we are applying a function to each row of our specified column. This leaves us with a new column of data with the function applied. 

In [4]:
# .apply() is a pandas method that applies a function across an index/column in a dataframe
yelpDataset['review_tokens'] = yelpDataset['review'].apply(lambda x: tokenize(x.lower()))

yelpDataset['review_alpha'] = yelpDataset['review_tokens'].apply(lambda x: onlyAlpha(x))
yelpDataset['review_nostops'] = yelpDataset['review_alpha'].apply(lambda x: noStop(x))
yelpDataset['review_spellCheck'] = yelpDataset['review_nostops'].apply(lambda x: spellCheck(x))
yelpDataset['review_lemmatized'] = yelpDataset['review_spellCheck'].apply(lambda x: lemmatize(x))

### From DataFrame to Separate Lists

Now that the data is clean, in order for our Feature Engineering and Vectorization to proceed we must turn our 2-dimensional DataFrame into two separate lists.

In [5]:
# First, we can turn our DataFrame column into a 1-dimensional DataFrame 
review = pd.DataFrame(data = yelpDataset['review_lemmatized'])

# Now, we can turn our 1-dimensional DataFrame into a list using Pandas' .tolist method
review = review['review_lemmatized'].tolist()

#The same is done to our sentiment column
sentiment = pd.DataFrame(data = yelpDataset['sentiment'])
sentiment = sentiment['sentiment'].tolist()

In [19]:
print(review)

['wow loved place', 'crust not good', 'not tasty texture nasty', 'stopped late may bank holiday rick steve recommendation loved', 'selection menu great price', 'getting angry want damn pho', 'honestly taste fresh', 'potato like rubber could tell made ahead time kept warmer', 'fry great', 'great touch', 'service prompt', 'would not go back', 'cashier care ever say still ended way overpriced', 'tried cape cod ravioli chicken cranberry mmmm', 'disgusted pretty sure human hair', 'shocked sign indicate cash', 'highly recommended', 'waitress little slow service', 'place not worth time let alone vega', 'not like', 'burrito blah', 'food amazing', 'service also cute', 'could care le interior beautiful', 'performed', 'right red velvet cake oath stuff good', 'never brought salad asked', 'hole wall great mexican street taco friendly staff', 'took hour get food table restaurant food luke warm sever running around like totally overwhelmed', 'worst salmon sashimi', 'also combo like burger fry beer de

## Feature Engineering and Vectorization

### Vectorize data, with 1- and 2- grams

Here we are importing a useful function that converts our data set of sentences into a large matrix of 0's and 1's. 

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

The matrix has each row representing a sentence and each column representing a word from the data set (no words are repeated). For each sentence, a 1 is placed in the columns of the words present in the sentence. If a word isn't in the sentence, a 0 is put in that column.

In [7]:
vectorizer = CountVectorizer(binary=True, lowercase=False)
# 'binary=false' would make the matrix count the frequency of a word in the sentence, instead of just marking its presence
# for some reason this wasn't working unless lowercase=false, some problem with the cleaned data I suppose

In [25]:
import pickle
print(type(vectorizer.vocabulary_)

{'wow': 1723, 'loved': 893, 'place': 1134, 'crust': 361, 'not': 1023, 'good': 661, 'tasty': 1522, 'texture': 1534, 'nasty': 1000, 'stopped': 1464, 'late': 843, 'may': 931, 'bank': 120, 'holiday': 737, 'rick': 1282, 'steve': 1456, 'recommendation': 1242, 'selection': 1346, 'menu': 950, 'great': 673, 'price': 1173, 'getting': 643, 'angry': 53, 'want': 1667, 'damn': 371, 'pho': 1122, 'honestly': 741, 'taste': 1518, 'fresh': 616, 'potato': 1163, 'like': 868, 'rubber': 1298, 'could': 334, 'tell': 1527, 'made': 906, 'ahead': 36, 'time': 1556, 'kept': 823, 'warmer': 1670, 'fry': 625, 'touch': 1576, 'service': 1356, 'prompt': 1187, 'would': 1721, 'go': 654, 'back': 111, 'cashier': 243, 'care': 235, 'ever': 515, 'say': 1326, 'still': 1458, 'ended': 496, 'way': 1681, 'overpriced': 1071, 'tried': 1589, 'cape': 231, 'cod': 290, 'ravioli': 1225, 'chicken': 268, 'cranberry': 347, 'mmmm': 971, 'disgusted': 430, 'pretty': 1172, 'sure': 1501, 'human': 758, 'hair': 695, 'shocked': 1368, 'sign': 1381, 'i

This next line could be used instead if we wished to make the program more sophisticated, but larger/slower. Instead of a column for every word, there would also be columns for every set of two words placed next to each other in the data set. You can change the 2 to any integer, but it makes the matrix exponentially larger.

In [8]:
#vectorizer = CountVectorizer(binary=True, lowercase=False, ngram_range=(1, 2))

Finally, we can simply call this function on our cleaned dataset 'phrases'.

In [9]:
vector = vectorizer.fit_transform(review)

In [17]:
print(vector)

  (0, 1134)	1
  (0, 893)	1
  (0, 1723)	1
  (1, 661)	1
  (1, 1023)	1
  (1, 361)	1
  (2, 1000)	1
  (2, 1534)	1
  (2, 1522)	1
  (2, 1023)	1
  (3, 1242)	1
  (3, 1456)	1
  (3, 1282)	1
  (3, 737)	1
  (3, 120)	1
  (3, 931)	1
  (3, 843)	1
  (3, 1464)	1
  (3, 893)	1
  (4, 1173)	1
  (4, 673)	1
  (4, 950)	1
  (4, 1346)	1
  (5, 1122)	1
  (5, 371)	1
  :	:
  (997, 1068)	1
  (997, 111)	1
  (997, 654)	1
  (997, 1721)	1
  (997, 1023)	1
  (998, 1017)	1
  (998, 1698)	1
  (998, 1504)	1
  (998, 1012)	1
  (998, 531)	1
  (998, 1610)	1
  (998, 1542)	1
  (998, 654)	1
  (998, 1556)	1
  (999, 453)	1
  (999, 1722)	1
  (999, 1164)	1
  (999, 1674)	1
  (999, 194)	1
  (999, 863)	1
  (999, 260)	1
  (999, 1313)	1
  (999, 501)	1
  (999, 1569)	1
  (999, 1556)	1


To use this with keras we need to first convert it to a numpy array.

In [22]:
# Change to a numpy array

data = vector.todense()
data = np.asarray(data)
print(type(data))

<class 'numpy.ndarray'>


In [24]:
print(data[0][1124])
print(len(data))

0
1000


Then we split the data up into the training, validation, and testing sets, at a 60:20:20 ratio.
We are careful to have an even number of positive and negative sentiments in each section.
Without staying even, our model would learn to guess, say, positive more often, simply because more reviews were positive!

In [11]:
# Split into train, test, and validate sets

x_train = np.concatenate([data[:300], data[-300:]])
y_train = np.concatenate([sentiment[:300], sentiment[-300:]])
x_val = np.concatenate([data[300:400], data[600:700]])
y_val = np.concatenate([sentiment[300:400], sentiment[600:700]])
x_test = np.concatenate([data[400:600]])
y_test = np.concatenate([sentiment[400:600]])
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(600, 1742)
(200, 1742)
(200, 1742)


## Model Architecture

### Model Creation 

This is where we finally make our neural network to process the vectorized data we made above. This process will consist of importing the libraries we need (keras in our instance), selecting the kind of model we wish to work with, creating the nueral network with it number of nodes and layers, selecting optimizer, loss, and metric functions, training, and then finally testing our data.

In [12]:
#Import the models library from the keras main library to use for our model
from keras import models

#Import the layers library from the keras main library, this will allow us to create a sequential neural network with sequential layers
from keras import layers


#Will make our model a sequential model, with sequences of dense layers
model = models.Sequential()

#Will set the first layer as a dense layer with 16 nodes, will use the activation function relu and fit our layer to the data
#Worth noting here that the number of layers and number of nodes are rather arbitrary, from our experience
#with this particular model, 1 dense layer with 16 nodes each allowed for maximal accuracy in the test data
#Will then output a single node dense layer that will output a continuous probability curve via the sigmoid function activator

model.add(layers.Dense(16, activation = 'relu', input_shape = (x_train.shape[1],)))
model.add(layers.Dense(1,  activation = 'sigmoid'))

Using TensorFlow backend.


### Model Compilation

Now that we have made our model, we must now choose what we want as our metric, our loss function, and our optimizer functions.

Very large degree of freedom here, many different loss functions to use, as well as metrics for model evaluation
and optimizer functions.

From our experience with this model, was more useful to use the following optimizer, loss, and metric functions,
but this can vary depending upon the project you are working on.

In [13]:
model.compile(optimizer = 'rmsprop',           #Set our optimizer function as "rmsprop"
              loss = 'binary_crossentropy',    #Set up our loss function as "binary cross entropy"
              metrics = ['accuracy'])          #Set up our metric function, will use "accuracy" here

### Model Training
Now that we have created our model and defined how we will evaluate our model and how it will optimize loss, will now train it.

To keep track of how our model is doing, will want to print out a history of each epoch to see how our model is improving, and if it is overfitting.

To train the model, must give our model the x_train and y_train data we set aside earlier, set a number of epochs we wish for the model to go through, the number of data entries per epoch, and (optionally), have it analyze a vlaidation set.

Again, worth noting here that the number of epochs and batch size are completely aribtrary and will vary form model to model, from our experience, we wished to maximize accuracy, so a greater number of epochs was desirable 

In [14]:
history = model.fit(x_train,                            #Input the x_train data
                       y_train,                         #Input the y_train data (features)
                       epochs=20,                       #Set our model to go through 20 epochs
                       batch_size=64,                   #Set our epoch batch size to 64 data entires
                       validation_data=(x_val, y_val))  #Have our model also go through the validation set, x_val, y_val

#To prevent overfitting, will plot the model accuracy against the validation accuracy and see where divergence occurs

#Will use the matplotlib.pyplot library to access the necessary plotting functions we will need to plot our output

Train on 600 samples, validate on 200 samples
Epoch 1/20
600/600 [==============================] - 2s 4ms/step - loss: 0.6895 - acc: 0.5817 - val_loss: 0.6895 - val_acc: 0.5650
Epoch 2/20
600/600 [==============================] - 0s 78us/step - loss: 0.6727 - acc: 0.6817 - val_loss: 0.6842 - val_acc: 0.6300
Epoch 3/20
600/600 [==============================] - 0s 52us/step - loss: 0.6571 - acc: 0.7933 - val_loss: 0.6796 - val_acc: 0.6350
Epoch 4/20
600/600 [==============================] - 0s 52us/step - loss: 0.6404 - acc: 0.8150 - val_loss: 0.6723 - val_acc: 0.6800
Epoch 5/20
600/600 [==============================] - 0s 78us/step - loss: 0.6218 - acc: 0.8600 - val_loss: 0.6638 - val_acc: 0.7050
Epoch 6/20
600/600 [==============================] - 0s 52us/step - loss: 0.6020 - acc: 0.8783 - val_loss: 0.6556 - val_acc: 0.7200
Epoch 7/20
600/600 [==============================] - 0s 52us/step - loss: 0.5807 - acc: 0.9000 - val_loss: 0.6454 - val_acc: 0.7500
Epoch 8/20
600/600 [====

In [15]:
import matplotlib.pyplot as plt

#Plot the model accuracy by calling history.history['acc']
plt.plot(history.history['acc'], label = "Model Accuracy")

#Add the validation accurayc to our plot by calling history.history['val_acc']
plt.plot(history.history['val_acc'], label = "Validation Accuracy")

#Add a grid to the plot for neatness in analysis
plt.grid()

#Add a legend to our plot
plt.legend()

#Adding a label to the x-axis 
plt.xlabel("Epochs")

#Adding a label to the y-axis
plt.ylabel("% Accuracy")

#Adding a limit to the x-axis range, want to make sure we get the full 0-20 epoch range
plt.xlim(0,20)

plt.show()

<Figure size 640x480 with 1 Axes>

### Model Testing

Now that we have tried to prevent overfitting the data and have trained it, let's see how well it does on our test data set.

In [16]:
#This step is as simple as calling the evaluate function from the keras models library with our x_test and y_test data sets
#as parameters to pass in
results = model.evaluate(x_test, y_test)

#We've tested it, now let's print out how accurate our model is on the testing data set
print ("Accuracy:", f"{(results[1]) * 100}%")

200/200 [==============================] - 0s 0us/step
Accuracy: 79.0%
